### Exemple 01

Comparative Analysis of Transcriptomic Response of *Escherichia coli* K-12 MG1655 to Nine Representative Classes of Antibiotics

doi: 10.1128/spectrum.00317-23

**Ref:** Bie L, Zhang M, Wang J, Fang M, Li L, Xu H, Wang M. Comparative Analysis of Transcriptomic Response of Escherichia coli K-12 MG1655 to Nine Representative Classes of Antibiotics. Microbiol Spectr. 2023 Feb 28;11(2):e0031723. doi: 10.1128/spectrum.00317-23. Epub ahead of print. PMID: 36853057; PMCID: PMC10100721.

- The exemple was done just for the IPM antibiotic

Obj: extract background and upregulated genes for functional enrichment analysis

In [1]:
import pandas as pd

In [5]:
#Get the data
df = pd.read_excel("spectrum.00317-23-s0002.xlsx")

In [6]:
new_header = df.iloc[0]

df = df[1:]  
df.columns = new_header
df

,Gene_id,chromosome,Strand,start,end,length,TET_1_readcount,TET_2_readcount,TET_3_readcount,MMC_1_readcount,...,biological_process,biological_process_description,cellular_component,cellular_component_description,molecular_function,molecular_function_description,KEGG AnnotInfo,KEGG url,Genename,Description
1,b0001,NC_000913.3,+,190,255,66,149,150,104,36,...,--,--,--,--,--,--,--,--,thrL,thr operon leader peptide
2,b0002,NC_000913.3,+,337,2799,2463,9016,7634,4637,493,...,GO:1901362//GO:0065007//GO:1901360//GO:0009058...,organic cyclic compound biosynthetic process//...,--,--,GO:0048037//GO:0036094//GO:0050661//GO:0050662...,cofactor binding//small molecule binding//NADP...,K12524 bifunctional aspartokinase / homoserine...,http://www.genome.jp/dbget-bin/www_bget?eco:b0002,thrA,Bifunctional aspartokinase/homoserine dehydrog...
3,b0003,NC_000913.3,+,2801,3733,933,2833,2917,1814,367,...,--,--,--,--,GO:0032555//GO:0008233//GO:0032549//GO:0032559...,purine ribonucleotide binding//peptidase activ...,K00872 homoserine kinase [EC:2.7.1.39] | (RefS...,http://www.genome.jp/dbget-bin/www_bget?eco:b0003,thrB,homoserine kinase
4,b0004,NC_000913.3,+,3734,5020,1287,4927,5295,3519,721,...,--,--,--,--,--,--,K01733 threonine synthase [EC:4.2.3.1] | (RefS...,http://www.genome.jp/dbget-bin/www_bget?eco:b0004,thrC,L-threonine synthase
5,b0005,NC_000913.3,+,5234,5530,297,178,186,146,101,...,--,--,--,--,--,--,no KO assigned | (RefSeq) yaaX; DUF2502 family...,http://www.genome.jp/dbget-bin/www_bget?eco:b0005,yaaX,DUF2502 family putative periplasmic protein
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4494,b4707,NC_000913.3,+,4019978,4020229,252,0,0,0,0,...,--,--,--,--,--,--,--,--,esrE,putative sRNA of unknown function
4495,b4708,NC_000913.3,+,279931,280104,174,6,9,7,3,...,GO:0008152//GO:0034660//GO:0009987//GO:0090304...,metabolic process//ncRNA metabolic process//ce...,--,--,GO:0016788//GO:0003674//GO:0003824//GO:0004540...,"hydrolase activity, acting on ester bonds//mol...",--,--,--,--
4496,b4712,NC_000913.3,+,3648063,3648146,84,65,68,65,35,...,--,--,--,--,--,--,--,--,agrA,inactive antisense sRNA
4497,b4713,NC_000913.3,+,3648294,3648377,84,3,4,0,1,...,--,--,--,--,--,--,--,--,agrB,antisense sRNA antitoxin for the DinQ toxin


In [7]:
#select the columns of interest
columns_to_keep = ['Gene_id',
    'IPM_readcount(IPMvsH2O)',
    'H2O_readcount(IPMvsH2O)',
    'log2FoldChange(IPMvsH2O)',
    'pval(IPMvsH2O)',
    'padj(IPMvsH2O)',
    'significant(IPMvsH2O)',
    'Genename'
]

dff

,Gene_id,IPM_readcount(IPMvsH2O),H2O_readcount(IPMvsH2O),log2FoldChange(IPMvsH2O),pval(IPMvsH2O),padj(IPMvsH2O),significant(IPMvsH2O),Genename
1,b0001,26.748774,35.445929,-0.40615,0.31839,0.53302,False,thrL
2,b0002,1013.776982,2605.517622,-1.3618,0.0,0.000004,DOWN,thrA
3,b0003,679.963515,1546.124088,-1.1851,0.000004,0.000048,DOWN,thrB
4,b0004,755.337278,1870.996555,-1.3086,0.0,0.000005,DOWN,thrC
5,b0005,103.473063,101.746877,0.024271,0.88865,0.98285,False,yaaX
...,...,...,...,...,...,...,...,...
4494,b4707,0,0,NaN,NaN,NaN,NaN,esrE
4495,b4708,0.301926,4.305983,-3.8341,0.027459,0.082698,False,--
4496,b4712,26.351368,35.955759,-0.44834,0.28212,0.49154,False,agrA
4497,b4713,0,1.441301,NaN,0.25867,0.46479,False,agrB


#### Backgraound genes

All genes expressed  (readcounts > 1)

In [8]:
#Select expressed genes for background
dff_background = dff[dff['IPM_readcount(IPMvsH2O)'] >= 1]
dff_background = dff_background.dropna()
len(dff_background)

4164

In [9]:
# Background formulation
background = []
for g in dff_background["Gene_id"]:
    if g not in background:
        background.append(g)

with open('background_ex1.txt', 'w') as file:
    for gene in background:
        file.write(f"{gene}\n")

In [10]:
print(len(dff_background["Gene_id"]))
with open("background_ex1.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()
print(len(lines))

4164
4164


#### Upregulated genes (genes of interest)

In the data presented in the article, genes are classified based on significance as follows: UP (when log2 fold change is at least 1), DOWN (when log2 fold change is at most -1), FALSE (when the gene does not meet the thresholds for upregulation or downregulation), and NA (when data is missing or annotation failed due to an error).

In [11]:
df_up = dff[dff['significant(IPMvsH2O)'] == 'UP'][['Gene_id', 'Genename', 'significant(IPMvsH2O)']]
df_up

,Gene_id,Genename,significant(IPMvsH2O)
21,b0023,rpsT,UP
23,b0025,ribF,UP
30,b0032,carA,UP
36,b0038,caiB,UP
37,b0039,caiA,UP
...,...,...,...
4311,b4502,yeiW,UP
4342,b4537,yecJ,UP
4346,b4542,yohO,UP
4372,b4573,--,UP


In [12]:
len(df_up[df_up["Genename"] == "--"])

7

In [13]:
#need to install ResPathExplorer: !pip install git+https://github.com/lais-carvalho/ResPathExplorer.git
from ResPathExplorer.mapper_KeggFunctions import get_gene_name_by_kegg_id

In [14]:
list_gene_ids = df_up[df_up['Genename'] == '--']['Gene_id'].tolist()
dict_g = {}
not_found_list = []

for id_g in list_gene_ids:
    id_go = "eco:" + id_g

    try:
        name = get_gene_name_by_kegg_id(id_go)

        if name:
            dict_g[id_g] = name
        else:
            dict_g[id_g] = ""

    except Exception as e:
        not_found_list.append(id_g) 
        continue

In [15]:
len(dict_g)

7

In [16]:
not_found_list

[]

In [17]:
keys_with_empty_values = [k for k, v in dict_g.items() if v == ""]
keys_with_empty_values

[]

In [18]:
#Upregulated genes
upregulated = []
for g in df_up["Gene_id"]:
    if g not in upregulated:
        upregulated.append(g)

with open('upregulated_ex1.txt', 'w') as file:
    for gene in upregulated:
        file.write(f"{gene}\n")

In [19]:
print(len(df_up["Gene_id"]))
with open("upregulated_ex1.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()
print(len(lines))

589
589


#### Get upregulated (genes of interest) gene names

For the CARDAnalysis and VFDBAnalysis, only genes with a registered name in the KEGG database were selected. Genes lacking an official gene name were excluded, even if they had a KEGG gene ID.

In [20]:
df_upf = df_up.copy()
mask = df_upf['Genename'] == '--'
df_upf.loc[mask, 'Genename'] = df_upf.loc[mask, 'Gene_id'].map(dict_g)

In [21]:
mask = df_upf['Genename'].isna()
df_upf.loc[mask, 'Genename'] = df_upf.loc[mask, 'Gene_id']

In [22]:
for d in df_upf[df_upf['Genename'].duplicated(keep=False)]["Gene_id"]:
    id_g = "eco:" + d
    name = get_gene_name_by_kegg_id(id_g)
    df_upf.loc[df_upf['Gene_id'] == d, 'Genename'] = name

In [23]:
df_upf[df_upf['Genename'].duplicated(keep=False)]

,Gene_id,Genename,significant(IPMvsH2O)


In [24]:
upregulated_GNames = []
for g in df_upf["Genename"]:
    if g not in upregulated_GNames:
        upregulated_GNames.append(g)

with open('upregulated_GNames_ex1.txt', 'w') as file:
    for gene in upregulated_GNames:
        file.write(f"{gene}\n")

In [25]:
print(len(df_upf["Genename"]))
with open("upregulated_GNames_ex1.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()
print(len(lines))

589
589


In [29]:
df_upf.to_excel("genenames.xlsx")